<a href="https://colab.research.google.com/github/github-rokon/CVPR/blob/main/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pillow pyheif

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'


import pyheif
import cv2
import random
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from PIL import Image


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Dataset'

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
def load_heic_image(image_path):
    # Read HEIC image using pyheif
    heif_file = pyheif.read(image_path)
    # Convert it into a format that Pillow can work with
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
        heif_file.mode,
        heif_file.stride,
    )
    return image

In [ ]:
# Load dataset with resizing images to the same dimensions
def load_dataset(dataset_path):
    images = []
    labels = []
    label_names = os.listdir(dataset_path)

    for index, name in enumerate(tqdm(label_names)):
        folder_path = os.path.join(dataset_path, name)
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)

            # Check if the file is a HEIC or regular image
            if image_name.lower().endswith('.heic'):
                try:
                    image = load_heic_image(image_path)  # Load HEIC image
                    image = np.array(image)
                except Exception as e:
                    print(f"Error loading HEIC image: {image_path} - {e}")
                    continue
            else:
                image = cv2.imread(image_path)  # Load non-HEIC image with OpenCV

            if image is not None:  # Check if the image was successfully loaded
                image = cv2.resize(image, (224, 224))  # Resize to 224x224
                images.append(image)
                labels.append(index)
            else:
                print(f"Warning: Could not load image {image_path}")

    images = np.array(images, dtype='float32') / 255.0  # Normalize
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=len(label_names))

    return images, labels, label_names # Return the images and labels

# Example usage
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Dataset' # Replace with the actual path to your dataset
images, labels, label_names = load_dataset(dataset_path) # Call the function and assign the returned values

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

  2%|▏         | 1/56 [00:00<00:49,  1.12it/s]

 80%|████████  | 45/56 [00:45<00:02,  3.93it/s]

Error loading HEIC image: /content/drive/MyDrive/Colab Notebooks/Dataset/SUN/IMG_6943.HEIC - Input is not a HEIF/AVIF file


100%|██████████| 56/56 [00:50<00:00,  1.11it/s]


In [ ]:


# Test different camera indices
for i in range(5):  # Try indices 0 to 4
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print(f"Camera found at index {i}")
        cap.release()

In [ ]:
# Build CNN model
def build_model(num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(num_classes, activation='softmax'),
    ])
    return model

model = build_model(num_classes=len(label_names))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 81s 7s/step - accuracy: 0.0267 - loss: 4.2143 - val_accuracy: 0.0230 - val_loss: 4.0004
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 78s 7s/step - accuracy: 0.1201 - loss: 3.7108 - val_accuracy: 0.1379 - val_loss: 3.7978
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 82s 7s/step - accuracy: 0.3894 - loss: 3.1916 - val_accuracy: 0.2529 - val_loss: 3.4023
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 81s 7s/step - accuracy: 0.5275 - loss: 2.3957 - val_accuracy: 0.3333 - val_loss: 2.8925
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 75s 7s/step - accuracy: 0.7283 - loss: 1.6130 - val_accuracy: 0.2644 - val_loss: 3.1442
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 81s 7s/step - accuracy: 0.8246 - loss: 0.9986 - val_accuracy: 0.3793 - val_loss: 2.6830
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 88s 7s/step - accuracy: 0.8997 - loss: 0.5539 - val_accuracy: 0.3563 - val_loss: 2.6129
Epoch 8/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 77s 7s/step - accuracy: 0.9716 - loss: 0.2871 - val_accuracy: 0.3793 - val_loss:

In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

# Function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

# Function to convert OpenCV Rectangle bounding box image into base64 byte string
def bbox_to_bytes(bbox_array):
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    bbox_PIL.save(iobuf, format='png')
    bbox_bytes = 'data:image/png;base64,{}'.format(str(b64encode(iobuf.getvalue()), 'utf-8'))
    return bbox_bytes

# Initialize the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
import cv2
import numpy as np
import time

# Function to preprocess the image for recognition
def preprocess_for_recognition(img):
    img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img_resized = img_resized / 255.0
    return np.expand_dims(img_resized, axis=0)

# Function to mark attendance
def mark_attendance(name):
    print(f"Marking attendance for {name}")

# Start streaming video from webcam
video_stream()

# Label for video
label_html = 'Capturing...'

# Initialize bounding box to empty
bbox = ''

# Dictionary to keep track of attendance
attendance_record = {}

# Delay between attendance markings (in seconds)
attendance_delay = 5

confidence_threshold = 0.7

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    img = js_to_image(js_reply["img"])

    bbox_array = np.zeros([480, 640, 4], dtype=np.uint8)

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    faces = face_cascade.detectMultiScale(gray)

    for (x, y, w, h) in faces:
        face_roi = img[y:y+h, x:x+w]

        face_for_recognition = preprocess_for_recognition(face_roi)

        prediction = model.predict(face_for_recognition)
        predicted_class_index = np.argmax(prediction)
        confidence = prediction[0][predicted_class_index]
        predicted_class = CATEGORIES[predicted_class_index]

        if confidence > confidence_threshold and predicted_class not in attendance_record:
            mark_attendance(predicted_class)
            attendance_record[predicted_class] = True

            time.sleep(attendance_delay)

        bbox_array = cv2.rectangle(bbox_array, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(bbox_array, f'{predicted_class} ({confidence:.2f})', (x + 2, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    bbox_array[:, :, 3] = (bbox_array.max(axis=2) > 0).astype(int) * 255
    bbox_bytes = bbox_to_bytes(bbox_array)
    bbox = bbox_bytes


<IPython.core.display.Javascript object>

In [ ]:


def real_time_recognition(model, label_names):
    # Initialize the webcam (use 0 as the parameter to select the default webcam)
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # For Windows

    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")



while True:
    ret, frame = cap.read()  # Read a frame from the webcam
    if not ret:
        break

    # Convert the frame to grayscale (Haar cascades work with grayscale images)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Draw a rectangle around each face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Extract the face region
        face_region = frame[y:y+h, x:x+w]

        # Preprocess the face region for prediction
        face = cv2.resize(face_region, (224, 224))
        face = np.expand_dims(face, axis=0)
        face = face / 255.0

        # Make prediction
        prediction = model.predict(face)
        predicted_class = label_names[np.argmax(prediction)]

        # Display the predicted class label
        cv2.putText(frame, predicted_class, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Show the frame with face rectangles and predicted class labels
    cv2.imshow('Real-time Face Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


    # Release the webcam and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Assuming 'model' is your trained model and 'label_names' is a list of class names
# real_time_recognition(model, label_names)